## Compare NIS Importer Template with Country Template

### Set Up Requirements

In [ ]:
pip install -r requirements.txt

### Set up Macros and Packages

In [ ]:
nis_download = r"C:\Users\ebradfield\OneDrive - Research Triangle Institute\Act East\NIS\Importer File Compare\sample data\DEM- Demography_20240826.xlsx"
country_download = r"C:\Users\ebradfield\OneDrive - Research Triangle Institute\Act East\NIS\Importer File Compare\sample data\DEM- Demography_20240829.xlsx"
summary_output= r"C:\Users\ebradfield\OneDrive - Research Triangle Institute\Act East\NIS\Importer File Compare\testreport.txt"
additions_deletions = r"C:\Users\ebradfield\OneDrive - Research Triangle Institute\Act East\NIS\Importer File Compare\additions_deletions.xlsx"

import pandas as pd 
import datacompy

### Read in Recently Downloaded Importer Template

In [ ]:
from_nis = pd.read_excel(io=f'{nis_download}', sheet_name ="Data", skiprows = 1)
from_nis.columns.values[0] = "Event UID"
from_nis.columns.values[1] = 'Organisation Unit*'
from_nis.columns.values[2] = 'Event Date'
from_nis.columns.values[3] = 'Latitude'
from_nis.columns.values[4] = 'Longitude'
from_nis.columns.values[5] = 'Status'
from_nis.drop(0, inplace=True)
print(from_nis)

### Generate Summary of Recently Downloaded Importer Template

In [ ]:
from_nis.nunique(axis=0, dropna=True)

### Read in Template from Country Team

In [ ]:
from_country = pd.read_excel(io=f'{country_download}', sheet_name ="Data", skiprows = 1)
from_country.columns.values[0] = "Event UID"
from_country.columns.values[1] = 'Organisation Unit*'
from_country.columns.values[2] = 'Event Date'
from_country.columns.values[3] = 'Latitude'
from_country.columns.values[4] = 'Longitude'
from_country.columns.values[5] = 'Status'
from_country.drop(0, inplace=True)
print(from_country)

### Generate Summary of Template from Country Team

In [ ]:
from_country.nunique(axis=0, dropna=True)

### Merge Dataframes and Identify Discrepancies

In [ ]:
compare = datacompy.Compare(from_nis, from_country, join_columns=['Event UID', 'Organisation Unit*'], df1_name='from_nis',
    df2_name='from_country')

print(compare.report(sample_count=1000000))

### Output Summary File

In [ ]:
print(compare.all_mismatch())
with open(summary_output, 'w', encoding='utf-8') as report_file:
    report_file.write(compare.report(sample_count=1000000))


### Write Additions and Deletions to Excel File

In [10]:
df1_new = compare.df1_unq_rows
df2_new = compare.df2_unq_rows

df1_new.to_excel(additions_deletions, sheet_name="From NIS", index = False)

with pd.ExcelWriter(additions_deletions, mode="a", engine="openpyxl") as writer:
    df2_new.to_excel(writer, sheet_name = "From Country", index = False)